# GigaSaiga

Продолжаем наш курс по генеративному ИИ. 

Сегодня мы научимся генерировать очень близкие к человеческим тексты стихов в стиле Пушкина.

Возьмём за основу проект GigaSaiga

https://huggingface.co/IlyaGusev/gigasaiga_lora

Это русскоязычная диалоговая система. За основу взята предобученная модель ai-forever/ruGPT-3.5-13B, и дообучена на дополнительных данных.

Вот код с примером использования

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/gigasaiga_lora"
DEFAULT_MESSAGE_TEMPLATE = "<s> {role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=2,
        bot_token_id=46787
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

# Интересно, что эта модель может сделать сходу?

In [2]:
inputs = ['Сочини стихотворение в стиле Пушкина, начинающееся со слов "Я помню чудное мгновение"']
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

Сочини стихотворение в стиле Пушкина, начинающееся со слов "Я помню чудное мгновение"
Я помню чудное мгновенье:
Передо мной явилась ты,
Как мимолетное виденье,
Как гений чистой красоты.

В тот миг душа моя ликовала,
И сердце трепетно стучало,
А взор твой нежный, как звезда,
Меня манил к себе всегда.

Но время быстро пролетело,
Исчезло счастье навсегда,
Лишь память о тебе осталась,
Как свет звезды далекой.

Так пусть же этот стих напомнит
О том прекрасном дне, когда
Мы встретились на этом свете,
Чтоб вечно помнить обо мне!




Первое четверостишье в точности повторяет оригинальное стихотворение, а вот дальше идёт новый текст, вполне осмысленный, и в стихотворной форме

# Посмотрим, что у этой модели внутри

 Модель квантизована на 8 бит и применён алгоритм LoRA

In [3]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50272, 5120)
        (wpe): Embedding(2048, 5120)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-39): 40 x GPT2Block(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): Linear8bitLt(
                in_features=5120, out_features=15360, bias=True
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDi

In [5]:
config.to_dict()

{'peft_type': 'LORA',
 'auto_mapping': None,
 'base_model_name_or_path': 'ai-forever/ruGPT-3.5-13B',
 'revision': None,
 'task_type': 'CAUSAL_LM',
 'inference_mode': True,
 'r': 8,
 'target_modules': ['c_attn'],
 'lora_alpha': 16,
 'lora_dropout': 0.1,
 'fan_in_fan_out': False,
 'bias': 'none',
 'modules_to_save': None,
 'init_lora_weights': True,
 'layers_to_transform': None,
 'layers_pattern': None}

In [6]:
generation_config

GenerationConfig {
  "bos_token_id": 2,
  "do_sample": true,
  "eos_token_id": 3,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.15,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}